In [1]:
from MIOFlow.models import make_model
import torch

In [2]:
batch_size = 4
feature_dim = 5
output_dim = 5

x = torch.randn(batch_size, feature_dim)
# m0 = torch.ones(batch_size, 1)
# xm = torch.cat([x, m0], dim=1)
t = torch.linspace(0, 1, 7)

model = make_model(feature_dims=feature_dim, output_dims=output_dim, which='ode_growth_rate')

In [3]:
xt, mt = model(x, t)

In [4]:
xt.shape, mt.shape

(torch.Size([4, 5]), torch.Size([4]))

In [5]:
xtseq, mtseq = model(x, t, return_whole_sequence=True)

In [6]:
xtseq.shape

torch.Size([7, 4, 5])

In [7]:
mtseq.shape

torch.Size([7, 4])

In [8]:
torch.nn.functional.softmax(mt, dim=-1) * mt.shape[-1]

tensor([1.1209, 0.9930, 0.9388, 0.9474], grad_fn=<MulBackward0>)

In [9]:
torch.exp(mt)/torch.exp(mt).sum(dim=0)

tensor([0.2802, 0.2482, 0.2347, 0.2369], grad_fn=<DivBackward0>)

In [10]:
torch.nn.functional.softmax(mtseq, dim=-1) * mtseq.shape[-1]

tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [1.0197, 0.9990, 0.9894, 0.9919],
        [1.0397, 0.9981, 0.9785, 0.9837],
        [1.0599, 0.9972, 0.9674, 0.9754],
        [1.0801, 0.9958, 0.9571, 0.9669],
        [1.1004, 0.9943, 0.9476, 0.9577],
        [1.1209, 0.9930, 0.9388, 0.9474]], grad_fn=<MulBackward0>)

In [11]:
xtseq.shape

torch.Size([7, 4, 5])

In [12]:
mtseq.unsqueeze(2).shape

torch.Size([7, 4, 1])

In [13]:
xmseq = torch.cat([xtseq, mtseq.unsqueeze(2)], dim=-1)
xmseq.shape

torch.Size([7, 4, 6])

In [14]:
dxdts = torch.stack([model.func(t[i], xmseq[i]) for i in range(len(t))])

In [15]:
dxdts.shape

torch.Size([7, 4, 6])

In [16]:
mt.unsqueeze(1).shape

torch.Size([4, 1])

In [17]:
model.func

ToyODE(
  (seq): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [18]:
xt.shape

torch.Size([4, 5])

In [19]:
mt.shape

torch.Size([4])

In [20]:
xm = torch.cat([xt, mt.unsqueeze(-1)], dim=-1)
dxdt = model.func(t[-1], xm)

In [21]:
dxdt[...,:-1].shape

torch.Size([4, 5])

In [22]:
mtseq.shape

torch.Size([7, 4])

In [23]:
xtseq.shape

torch.Size([7, 4, 5])

In [24]:
t.shape

torch.Size([7])

In [25]:
t[[0,1,2],...]

tensor([0.0000, 0.1667, 0.3333])

In [26]:
mt.shape

torch.Size([4])

In [27]:
(dxdt * mt.unsqueeze(-1)).shape

torch.Size([4, 6])

In [28]:
dxdts.shape

torch.Size([7, 4, 6])

In [29]:
mtseq.shape

torch.Size([7, 4])

In [30]:
(dxdts * mtseq.unsqueeze(-1)).shape

torch.Size([7, 4, 6])

In [31]:
torch.nn.functional.softmax(mtseq, dim=-1).shape

torch.Size([7, 4])

In [32]:
mtseq.shape[-1]

4

In [33]:
torch.nn.functional.softmax(mt, dim=-1).shape

torch.Size([4])

In [34]:
mt.shape[-1]

4

In [35]:
def func_end_0(t, x):
    xm = model.func(t, x)
# mt[[0,2,4,6,8]] = 0.
model.func(t[-1], xm).shape

torch.Size([4, 6])

In [36]:
xm[...,-1].shape

torch.Size([4])

In [37]:
xm.shape

torch.Size([4, 6])

In [38]:
xmseq.shape

torch.Size([7, 4, 6])

In [40]:
xm[[0,2],-1] = 0.

In [41]:
xm

tensor([[ 0.1330, -1.2687,  0.3459, -2.1293, -0.2753,  0.0000],
        [ 0.6631,  0.6218, -0.1540, -0.3031, -1.0497,  0.8914],
        [-0.8940,  0.5662,  0.8082, -0.6803, -0.1911,  0.0000],
        [ 0.3448,  1.1493,  0.5407,  0.4708,  0.7046,  0.8444]],
       grad_fn=<CopySlices>)

In [42]:
dxmdt = model.func(t[-1], xm)

In [43]:
dxmdt

tensor([[ 0.0507, -0.2467,  0.0696, -0.0077,  0.0235,  0.0000],
        [ 0.0895,  0.0575,  0.1896, -0.0868, -0.0061, -0.0919],
        [-0.0329, -0.0656, -0.2477,  0.0865,  0.2367,  0.0000],
        [-0.0359, -0.2023, -0.2403,  0.0443,  0.1565, -0.1530]],
       grad_fn=<CopySlices>)